### Обучение TextToLabel

In [132]:
import pandas as pd

files: list[str] = [
    "dataset/transcription/transcribed.csv",
    # "dataset/transcription/hr_bot_clear.json",
    # "dataset/transcription/hr_bot_noise.json",
    # "dataset/transcription/hr_bot_synt.json",
    # "dataset/transcription/luga.json",
]

li = []
for filename in files:
    df = pd.read_csv(filename)
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)
df = df.sample(frac=1)
df.head()

,Unnamed: 0,audio_filepath,id,text,label,attribute,transcription
802,2412,dataset/hr_bot_clear/4a40e4d7-76ff-11ee-a117-c...,4a40e4d7-76ff-11ee-a117-c09bf4619c03,тише,18,-1,тише
263,1401,dataset/hr_bot_clear/535f131b-76ff-11ee-a72f-c...,535f131b-76ff-11ee-a72f-c09bf4619c03,осадить на четырнадцать вагонов,4,14,а садив на четырнадцать вагонов
1431,2104,dataset/hr_bot_clear/7932987b-76ff-11ee-9612-c...,7932987b-76ff-11ee-9612-c09bf4619c03,начать осаживание,3,-1,начайте саживанье
1247,1142,dataset/hr_bot_clear/5b0e7795-76ff-11ee-9bcf-c...,5b0e7795-76ff-11ee-9bcf-c09bf4619c03,осадить на пятнадцать вагонов,4,15,а садить на пятнадцать вагонов
2282,902,dataset/hr_bot_noise/568e86b7-76fe-11ee-bf4d-c...,568e86b7-76fe-11ee-bf4d-c09bf4619c03,протянуть на два вагона,10,2,протянуть на два вагона


In [133]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

# vectorizer = HashingVectorizer(
#     lowercase=True, n_features=2**10, stop_words="english"
# )
vectorizer = TfidfVectorizer(
    lowercase=True, sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english"
)
# classifier = RandomForestClassifier(n_estimators=500, random_state=42)
classifier = MultinomialNB()

pipeline = Pipeline(
    [
        ("tfidf", vectorizer),
        ("classifier", classifier),
    ]
)

In [134]:
from label2id import id2label

X = df["transcription"]
y = df["label"].apply(id2label).values

In [135]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.5, min_df=5, stop_words='english',
                                 sublinear_tf=True)),
                ('classifier', MultinomialNB())])

In [136]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

y_pred = pipeline.predict(X_test)
f1_weighted = f1_score(y_test, y_pred, average='weighted')
print(f"F1: {f1_weighted:.2f}")

scores = cross_val_score(pipeline, X, y, cv=5)
print('Cross val:', scores.mean())

F1: 0.87
Cross val: 0.9013182674199622


In [137]:
import joblib
joblib.dump(pipeline, "trained/text2label.pkl")

['trained/text2label.pkl']